In [3]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [5]:
G = nx.read_gexf('networks/AM-RM-16-singleDi.gexf')

In [6]:
graph_data = pd.read_csv('networks/AM-RM-16-singleDi.csv')

# Calculating Frequency Based Transfer Oppurtunities

In [8]:
degree_df = pd.DataFrame([[key, value] for key, value in dict(G.degree(weight = 'freq')).items()], columns = ['node', 'degree'])

In [9]:
degree_df.sort_values(by = 'degree', ascending = False)

,node,degree
60,228,311
16,116,309
64,304,307
11,111,264
390,13444423,251
...,...,...
3521,13452472,3
3092,13467390,2
3005,13466488,2
3523,13458080,2


In [10]:
graph_data

,INT_ID_o,INT_ID_d,freq,scaled_freq,link
0,100,101,26,0.3250,100-101
1,100,151,21,0.2625,100-151
2,100,13467163,13,0.1625,100-13467163
3,100,13467239,6,0.0750,100-13467239
4,100,13467593,6,0.0750,100-13467593
...,...,...,...,...,...
7810,30113231,13446581,4,0.0500,30113231-13446581
7811,30113231,13446900,4,0.0500,30113231-13446900
7812,30113634,13446075,11,0.1375,30113634-13446075
7813,30113634,13446293,11,0.1375,30113634-13446293


In [12]:
stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [13]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,cost,route_short_name,direction_id
0,32828689,1,1,5,15,13466640.0,2,5,16,13466531.0,1.0,506,0
1,32828689,2,2,5,16,13466531.0,3,5,16,13466677.0,0.0,506,0
2,32828689,3,3,5,16,13466677.0,4,5,17,13466754.0,1.0,506,0
3,32828689,4,4,5,17,13466754.0,5,5,18,13466699.0,1.0,506,0
4,32828689,5,5,5,18,13466699.0,6,5,19,14043045.0,1.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213714,32970488,6,7,3,4,13461421.0,8,3,4,13461602.0,0.0,306,1
1213715,32970488,7,8,3,4,13461602.0,9,3,5,13461912.0,1.0,306,1
1213716,32970488,8,9,3,5,13461912.0,10,3,5,13462284.0,0.0,306,1
1213717,32970488,9,10,3,5,13462284.0,11,3,6,13462427.0,1.0,306,1


In [17]:
stop_times_od = stop_times_od[stop_times_od['hr_o'] == 4]

In [18]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,cost,route_short_name,direction_id
79990,32835650,1,1,4,50,227.0,3,4,52,13455771.0,2.0,102,1
79991,32835650,2,3,4,52,13455771.0,4,4,53,13455635.0,1.0,102,1
79992,32835650,3,4,4,53,13455635.0,5,4,54,13455585.0,1.0,102,1
79993,32835650,4,5,4,54,13455585.0,6,4,55,13455403.0,1.0,102,1
79994,32835650,5,6,4,55,13455403.0,7,4,55,13455269.0,0.0,102,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212812,32970467,18,19,4,56,13463573.0,20,4,57,13463705.0,1.0,306,1
1212813,32970467,19,20,4,57,13463705.0,21,4,58,13463796.0,1.0,306,1
1212814,32970467,20,21,4,58,13463796.0,22,4,58,13463892.0,0.0,306,1
1212815,32970467,21,22,4,58,13463892.0,23,4,59,13464004.0,1.0,306,1


In [20]:
graph_data = stop_times_od[['trip_id', 'INT_ID_o', 'INT_ID_d','route_short_name', 'direction_id', 'cost']]
graph_data

,trip_id,INT_ID_o,INT_ID_d,route_short_name,direction_id,cost
79990,32835650,227.0,13455771.0,102,1,2.0
79991,32835650,13455771.0,13455635.0,102,1,1.0
79992,32835650,13455635.0,13455585.0,102,1,1.0
79993,32835650,13455585.0,13455403.0,102,1,1.0
79994,32835650,13455403.0,13455269.0,102,1,0.0
...,...,...,...,...,...,...
1212812,32970467,13463573.0,13463705.0,306,1,1.0
1212813,32970467,13463705.0,13463796.0,306,1,1.0
1212814,32970467,13463796.0,13463892.0,306,1,0.0
1212815,32970467,13463892.0,13464004.0,306,1,1.0


In [21]:
route_dir_int = graph_data[['INT_ID_o', 'route_short_name', 'direction_id']].drop_duplicates()
route_dir_int = route_dir_int.rename(columns = {'INT_ID_o':'node'})

In [22]:
route_dir_int = route_dir_int.append(graph_data[['INT_ID_d', 'route_short_name', 'direction_id']].drop_duplicates().rename(
    columns = {'INT_ID_d':'node'})).drop_duplicates()
route_dir_int

,node,route_short_name,direction_id
79990,227.0,102,1
79991,13455771.0,102,1
79992,13455635.0,102,1
79993,13455585.0,102,1
79994,13455403.0,102,1
...,...,...,...
1200972,203.0,35,0
1202985,13446896.0,35,1
1210833,13463074.0,35,1
1211868,225.0,306,0


In [23]:
route_dir = route_dir_int
route_dir_int = route_dir_int.groupby('node').count().reset_index().rename(columns = {'route_short_name':'route-direction'})[['node', 'route-direction']]

In [24]:
route_dir_int

,node,route-direction
0,100.0,3
1,101.0,4
2,102.0,4
3,103.0,2
4,104.0,4
...,...,...
1744,30036276.0,1
1745,30070005.0,3
1746,30072982.0,2
1747,30109241.0,4


In [25]:
degree_df['node'] = degree_df['node'].astype(int)
connect_df = degree_df.merge(route_dir_int)
connect_df['route-direction'] = (connect_df['route-direction'] - 2)/2
connect_df['degree'] = connect_df['degree']/2

In [26]:
connect_df['route-direction'] = np.where(connect_df['route-direction'] < 0, 0, connect_df['route-direction'])
connect_df['transfer-poss'] = connect_df['route-direction'] * connect_df['degree']

In [28]:
int_stops = pd.read_csv(simplify_dir + 'int_stop.csv')
connect_df = connect_df.merge(int_stops, left_on = ['node'], right_on = ['INT_ID']
                             )[['INT_ID', 'degree', 'route-direction', 'transfer-poss', 'int_lon', 'int_lat']]

In [29]:
connect_df.to_csv('networks/EM/EM_transfer-poss.csv', index = False)

# Conversion to Network Structural Connectivity

In [30]:
ward_int = pd.read_csv('GIS/int_wards.csv')

In [45]:
multiple_edges = graph_data[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()

In [46]:
multiple_freq = graph_data[['INT_ID_o', 'INT_ID_d', 'route_short_name']]
multiple_freq['freq'] = True
multiple_freq = multiple_freq.groupby(['INT_ID_o', 'INT_ID_d', 'route_short_name']).count().reset_index()

/Users/Rick/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Example Iteration

In [33]:
i = 15

In [34]:
ward_int_list = list(ward_int[ward_int['AREA_SHORT_CODE'] == i]['INT_ID'])
multiple_edges_ward = multiple_edges[multiple_edges['INT_ID_o'].isin(ward_int_list)]
multiple_edges_ward = multiple_edges_ward[multiple_edges_ward['INT_ID_d'].isin(ward_int_list)]

In [35]:
multiple_freq_ward = multiple_freq[multiple_freq['INT_ID_o'].isin(ward_int_list)]
multiple_freq_ward = multiple_freq_ward[multiple_freq_ward['INT_ID_d'].isin(ward_int_list)]

In [36]:
multiple_edges_ward = multiple_edges_ward.groupby(['INT_ID_o', 'INT_ID_d']).count()
multiple_edges_ward = multiple_edges_ward.reset_index()
multiple_edges_ward['route_short_name'] = multiple_edges_ward['route_short_name']-1

In [37]:
multiple_edges_ward = multiple_edges_ward.rename(columns = {'route_short_name':'count_routes'})

In [38]:
multiple_freq_ward = multiple_freq_ward.merge(multiple_edges_ward, how = 'left')
multiple_freq_ward = multiple_freq_ward[multiple_freq_ward['count_routes']>0]

In [39]:
e_multiple = multiple_freq_ward.sort_values(by = 'freq', ascending = False).drop_duplicates(
    subset = ['INT_ID_o', 'INT_ID_d'], keep = 'first')['freq'].sum()/2
e_multiple

26.0

In [40]:
table = []

for i in range(1, 26):
    
    # filtering wards
    ward_int_list = list(ward_int[ward_int['AREA_SHORT_CODE'] == i]['INT_ID'])
    multiple_edges_ward = multiple_edges[multiple_edges['INT_ID_o'].isin(ward_int_list)]
    multiple_edges_ward = multiple_edges_ward[multiple_edges_ward['INT_ID_d'].isin(ward_int_list)]

    multiple_freq_ward = multiple_freq[multiple_freq['INT_ID_o'].isin(ward_int_list)]
    multiple_freq_ward = multiple_freq_ward[multiple_freq_ward['INT_ID_d'].isin(ward_int_list)]

    multiple_edges_ward = multiple_edges_ward.groupby(['INT_ID_o', 'INT_ID_d']).count()
    multiple_edges_ward = multiple_edges_ward.reset_index()
    multiple_edges_ward['route_short_name'] = multiple_edges_ward['route_short_name']-1
    multiple_edges_ward = multiple_edges_ward.rename(columns = {'route_short_name':'count_routes'})

    multiple_freq_ward = multiple_freq_ward.merge(multiple_edges_ward, how = 'left')
    multiple_freq_ward = multiple_freq_ward[multiple_freq_ward['count_routes']>0]
    
    # checking for multiple edges
    e_multiple = multiple_freq_ward.sort_values(by = 'freq', ascending = False).drop_duplicates(
        subset = ['INT_ID_o', 'INT_ID_d'], keep = 'first')['freq'].sum()/2
    
    # calculating number of transfer stations
    transfer_stop = len(connect_df[(connect_df['INT_ID'].isin(ward_int_list)) &
                                   (connect_df['route-direction'] > 0)])
    
    ward_connect = connect_df[connect_df['INT_ID'].isin(ward_int_list)]
    
    # other stats measures
    skew = stats.skew(ward_connect['transfer-poss'])
    kurt = stats.kurtosis(ward_connect['transfer-poss'])
    
    # total transfer possibilities
    tot_transfer = ward_connect['transfer-poss'].sum()
    
    
    if transfer_stop > 0:
        connectivity = (tot_transfer - e_multiple)/transfer_stop
    else:
        print(i)
        connectivity = 0
        
    table.append([i, skew, kurt, tot_transfer, e_multiple, transfer_stop, connectivity])

In [41]:
connectivity_df = pd.DataFrame(table, columns = ['AREA_SHORT_CODE', 'skew', 'kurtosis', 
                               'total_transfer_oppurtunities', 'multiple_edges', 
                               'num_transfer_stops', 'connectivity'])

In [42]:
connectivity_df = connectivity_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates())
connectivity_df.to_csv('networks/EM/EM_connectivity.csv', index = False)
connectivity_df = connectivity_df.rename(columns = {'AREA_SHORT_CODE': 'Ward', 'AREA_NAME': 'Ward Name'})
connectivity_df = connectivity_df[['Ward', 'Ward Name', 'skew', 'kurtosis', 
                               'total_transfer_oppurtunities', 'multiple_edges', 
                               'num_transfer_stops', 'connectivity']]


In [43]:
connectivity_df.sort_values(by = 'connectivity')

,Ward,Ward Name,skew,kurtosis,total_transfer_oppurtunities,multiple_edges,num_transfer_stops,connectivity
0,1,Etobicoke North,3.131200,10.150454,860.00,24.0,38,22.000000
1,2,Etobicoke Centre,3.886688,15.942457,431.25,6.0,18,23.625000
16,17,Don Valley North,3.282741,10.214203,796.25,27.0,29,26.525862
24,25,Scarborough-Rouge Park,3.398001,12.879971,724.25,19.0,25,28.210000
6,7,Humber River-Black Creek,3.876694,16.837721,1648.75,62.0,54,29.384259
22,23,Scarborough North,2.899133,8.648287,1125.00,36.5,34,32.014706
23,24,Scarborough-Guildwood,2.401457,5.116379,982.00,29.5,26,36.634615
18,19,Beaches-East York,2.997754,9.534306,1000.00,38.5,26,36.980769
4,5,York South-Weston,2.918869,8.980388,2238.75,64.5,56,38.825893
5,6,York Centre,3.184596,10.134387,2050.25,43.0,50,40.145000


## Skewness and Kurtosis

High skewness and kurtosis means there is a huge variation in the redundant oppurtunities per stop. This means that wards at the top of the list likely have a few highly connected stops with many stops having no redundancy.

Wards at the bottom of the list have similar levels of redundancy throughout (either consistently high, or consistently low). They are consistently connected and tend to have grid patterns.

In [44]:
connectivity_df.sort_values(by = 'skew', ascending = False)

,Ward,Ward Name,skew,kurtosis,total_transfer_oppurtunities,multiple_edges,num_transfer_stops,connectivity
2,3,Etobicoke-Lakeshore,5.773637,35.969364,618.00,5.0,15,40.866667
17,18,Willowdale,5.445273,31.683520,947.50,23.0,14,66.035714
3,4,Parkdale-High Park,4.697960,26.177858,1920.00,37.5,34,55.367647
11,12,Toronto-St. Paul's,4.390739,22.252109,1874.75,31.0,30,61.458333
1,2,Etobicoke Centre,3.886688,15.942457,431.25,6.0,18,23.625000
6,7,Humber River-Black Creek,3.876694,16.837721,1648.75,62.0,54,29.384259
13,14,Toronto-Danforth,3.757501,17.040019,1457.25,17.0,22,65.465909
19,20,Scarborough Southwest,3.691808,14.173603,1894.00,42.5,32,57.859375
7,8,Eglinton-Lawrence,3.600245,13.844445,1838.75,53.0,38,46.993421
24,25,Scarborough-Rouge Park,3.398001,12.879971,724.25,19.0,25,28.210000
